In [412]:
import os
import re
import time
import tiktoken
from dotenv import load_dotenv
from fpdf import FPDF
from langchain_community.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic

In [139]:
load_dotenv()

encoding = tiktoken.get_encoding("cl100k_base")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

In [68]:
# TODO:
#   step 1: Structuring the transcripts: ???? Переложить в таблицу
#   step 2: Performing initial coding: 
#   step 3: need res prev step and then Finding topics and themes: 
#   step 4: Calculating the frequency of problems mentioned
#   step 5: need res prev step and then Grouping problems into categories 
#   step 6: need res prev step and then Making insights and forming hypotheses 

## Functions

In [389]:
def get_part_answer_llm(input_string: str = None, tag: str = None) -> str:
    pattern = re.compile(rf"<{tag}>(.*?)<\/{tag}>", re.DOTALL)
    matches = pattern.search(input_string)

    results = ""
    if matches:
        extracted_data = matches.group(1)
        results += extracted_data

    text = re.sub(r'^\s*-\s*', '', results, flags=re.MULTILINE)
    text = re.sub(r'\n', '. ', text)
    text = re.sub(r'\s*\.\s*', '. ', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip(' .')
    
    return text.strip()

## Load files

In [140]:
loader = DirectoryLoader("../data/demo", glob="*.txt")
documents = loader.load()

for num, doc in enumerate(documents):
    print(
        f"index: {num}, file: {doc.metadata['source'][-15:]}, length: {len(doc.page_content)}, tokens: {len(encoding.encode(doc.page_content))}.")

index: 0, file: interview_2.txt, length: 33951, tokens: 15518.
index: 1, file: interview_1.txt, length: 36025, tokens: 16647.


In [141]:
interview_1 = documents[0].page_content
interview_2 = documents[1].page_content

## Model

In [411]:
llm_haiku = ChatAnthropic(api_key=ANTHROPIC_API_KEY, model="claude-3-haiku-20240307", max_tokens=4096, temperature=0.0)
llm_haiku_t = ChatAnthropic(api_key=ANTHROPIC_API_KEY, model="claude-3-haiku-20240307", max_tokens=4096, temperature=0.5)

## For One Interview

In [359]:
def asks_llm(llm: None, raw_data: list[str] = None, text_template: str = None, input_variables: list[str] = None):
    """
    
    :param llm: 
    :param raw_interview: 
    :param text_template: 
    :param input_variables: 
    :param input_chain: 
    :return: 
    """

    upd = "\nAttention, this is important! Always send the full text of the response in Russian only. Thanks!"

    prompt = PromptTemplate(input_variables=input_variables, template=text_template + upd)
    chain = prompt | llm

    input_chain = {}
    for var, data in zip(input_variables, raw_data):
        input_chain[var] = data
         
    answer = chain.invoke(input_chain)
    return answer

In [ ]:
# answer_1 = asks_llm(
#     llm=llm_haiku, raw_data=[interview_1],
#     text_template="""
#         Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
# 
#         Step-by-step instructions:
#         1. Identify the main sections or phases of the interview, for example, introduction, general questions, main topics, concluding remarks. (In <section_main_phases> tags.)
#         2. Break down each transcript into these sections, marking the beginning and end of each part. (In <section_transcripts> tags.)
#         3. Create a standardized template for  <main_sections>structuring</main_sections> the transcripts consistently. (In <section_table> tags.)""",
#     input_variables=["raw_interview"],
# )

In [293]:
# answer_1_section_table = get_part_answer_llm(input_string=answer_1.content, tag="section_table")
# print(answer_1_section_table)

In [391]:
# answer_2 = asks_llm(
#     llm=llm_haiku, raw_data=[interview_1],
#     text_template="""
#         Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
#         
#         Step-by-step instructions:
#         1. Read through each interviews carefully and highlight or underline key phrases, words, or sentences that capture important ideas, concepts, or experiences. (In <section_key_phrases> tags.)
#         2. Assign preliminary codes or labels to these highlighted segments, using short descriptive phrases or words that summarize the main idea. (In <section_codes> tags.)""",
#     input_variables=["raw_interview"],
# )

In [392]:
# answer_2_section_codes = get_part_answer_llm(input_string=answer_2.content, tag="section_codes")
# print(answer_2_section_codes)

Профиль респондента. Сфера деятельности и специализация. Форматы сделок. Должностные обязанности. Обязательное начальное обучение. Постоянное повышение квалификации. Источники информации. Предпочтения в форматах обучения. Важность обратной связи и групповой работы. Мнение о платности обучения. Потребность в обучении по инвестициям


In [364]:
# answer_3 = asks_llm(
#     llm=llm_haiku, raw_data=[interview_1, answer_2_section_codes],
#     text_template="""
#         Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
#         
#         Step-by-step instructions:
#         1. Look through the thematics codes <interview_code>{interview_code}</interview_code> and find patterns, similarities, or relationships among them. (In <section_relationships> tags.)
#         2. Group related thematics codes together into broader categories or themes that capture the main topics discussed in the interviews. (In <section_group> tags.)
#         3. Identify any sub-themes within each main theme to provide a more detailed understanding of the topics. (In <section_sub-themes> tags.)
#         4. Create a diagram of the relationships between topics and subtopics to illustrate their relationship. (In <section_map> tags.)""",
#     input_variables=["raw_interview", "interview_code"]
# )

In [365]:
# print(answer_3.content)

<section_relationships>
В интервью можно выделить следующие взаимосвязи между тематическими кодами:

- Профиль респондента и сфера деятельности/специализация: Светлана имеет педагогическое образование, но перешла в сферу недвижимости, чтобы зарабатывать больше денег.
- Начальное обучение и его значение: Светлана прошла обязательное обучение в агентстве недвижимости, чтобы получить необходимые знания и навыки для работы риелтором.
- Непрерывное обучение и повышение квалификации: Светлана продолжает регулярно проходить различные курсы и тренинги, чтобы быть в курсе изменений на рынке и повышать свою компетентность.
- Источники информации и их оценка: Светлана активно использует телеграм-каналы, рассылки от партнеров и другие источники для получения актуальной информации о рынке недвижимости.
- Обратная связь и групповая работа в обучении: Светлана отмечает важность обратной связи от преподавателей и возможности обсуждения материала в группе во время обучения.
</section_relationships>

<s

In [382]:
# answer_4 = asks_llm(
#     llm=llm_haiku, raw_data=[interview_1],
#     text_template="""
#         Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
#         
#         Step-by-step instructions:
#         1. Make a list of all the specific problems mentioned by the respondents. (In the <section_problems> tags)
#         2. Count how many times each problem has been mentioned. (In the <section_problem_count> tags.)
#         3. Create a table to display the frequency of occurrence of each problem in descending order. (In the <section_problem_table> tags.)
#         4. Create a table of the frequency of occurrence of each problem by dividing the number of dimensions of the problem by the total number of problems. In the table, be sure to show the problem itself, how many times it occurs, its share relative to all problems, and the total number of problems. (In the <section_problem_table_normalize> tags).
#         """,
#     input_variables=["raw_interview"]
# )

In [383]:
# print(answer_4.content)

<section_problems>
- Недостаток обучения по инвестициям в недвижимость
- Большое количество участников в группах на курсах (более 15 человек)
- Недостаток обучения по юридическим аспектам в сфере недвижимости
</section_problems>

<section_problem_count>
- Недостаток обучения по инвестициям в недвижимость: 1
- Большое количество участников в группах на курсах (более 15 человек): 1
- Недостаток обучения по юридическим аспектам в сфере недвижимости: 1
</section_problem_count>

<section_problem_table>
| Проблема | Количество упоминаний |
| --- | --- |
| Недостаток обучения по инвестициям в недвижимость | 1 |
| Большое количество участников в группах на курсах (более 15 человек) | 1 |
| Недостаток обучения по юридическим аспектам в сфере недвижимости | 1 |
</section_problem_table>

<section_problem_table_normalize>
| Проблема | Количество упоминаний | Доля от общего числа проблем | Всего проблем |
| --- | --- | --- | --- |
| Недостаток обучения по инвестициям в недвижимость | 1 | 0.33 | 3 |

In [393]:
# answer_4_section_problems = get_part_answer_llm(input_string=answer_4.content, tag="section_problems")
# print(answer_4_section_problems)

Недостаток обучения по инвестициям в недвижимость. Большое количество участников в группах на курсах (более 15 человек). Недостаток обучения по юридическим аспектам в сфере недвижимости


In [399]:
# answer_5 = asks_llm(
#     llm=llm_haiku, raw_data=[interview_1, answer_4_section_problems],
#     text_template="""
#         Please carefully read the interview provided to you and the problems identified in it. Follow the step-by-step instructions below: 
#         <interview>{raw_interview}</interview> and <interview_problems>{interview_problems}</interview_problems> 
#         
#         Step-by-step instructions:
#         1. Review the list of problems and look for commonalities or shared characteristics among them. (In <section_common_features> tags.)
#         2. Develop a set of categories or problems types that can encompass the various issues mentioned. (In <section_problems_categories> tags.)
#         3. Assignment each problem (<problems>) to one or more relevant categories. (In <section_map> tags.)
#         4. Create a summary table or matrix showing the problems grouped by category. (In <section_result> tags.).
#         """,
#     input_variables=["raw_interview", "interview_problems"]
# )

In [400]:
# print(answer_5.content)

<section_common_features>
Основные общие черты проблем, выявленных в интервью:
1. Недостаток специализированного обучения по ключевым аспектам деятельности риелтора, таким как инвестиции в недвижимость и юридические вопросы.
2. Большое количество участников в группах на курсах, что затрудняет эффективное усвоение материала.
</section_common_features>

<section_problems_categories>
Основные категории проблем:
1. Недостаток обучения по специализированным темам
2. Неоптимальный формат обучения (большие группы)
3. Недостаток обучения по юридическим аспектам
</section_problems_categories>

<section_map>
1. Недостаток обучения по инвестициям в недвижимость - категория 1
2. Большое количество участников в группах на курсах (более 15 человек) - категория 2
3. Недостаток обучения по юридическим аспектам в сфере недвижимости - категория 3
</section_map>

<section_result>
Сводная таблица проблем, выявленных в интервью:

| Категория проблем | Описание проблемы |
| --- | --- |
| 1. Недостаток обуче

In [401]:
# answer_5_section_problems_categories = get_part_answer_llm(input_string=answer_5.content, tag="section_problems_categories")
# print(answer_5_section_problems_categories)

Основные категории проблем:. 1. Недостаток обучения по специализированным темам. 2. Неоптимальный формат обучения (большие группы). 3. Недостаток обучения по юридическим аспектам


In [402]:
# answer_6 = asks_llm(
#     llm=llm_haiku_t, raw_data=[interview_1, answer_5_section_problems_categories],
#     text_template="""
#         Please carefully read the interview provided to you and the categories of problems identified in it. Follow the step-by-step instructions below: 
#         <interview>{raw_interview}</interview> and <interview_problems>{interview_problems_categories}</interview_problems> 
#         
#         Step-by-step instructions:
#         1. Analyze the topics, frequency of problems and their categories (<problems>). (In the <section_reflections> tags). 
#         2. Analyze what these <reflection_sections> results say about the respondent's experience, views, or difficulties. (In the <section_results> tags). 
#         3. Pay attention to any unexpected patterns, connections, or contrasts in the data. Share these observations. (In the <section_unexpected> tags).
#         4. Develop preliminary explanations or hypotheses about why certain topics or issues have arisen and how they may relate to each other or to broader contextual factors. (In the <section_hypothesis> tags).
#         5. Consider alternative explanations and counterexamples to clarify your hypothesis about <section_hypothesis>. (In the <section_alternatives> tags.)
#         6. Identify areas where additional research or analysis may be needed to validate your ideas in the <section_alternatives>. (In the <section_additionals> tags).
#         """,
#     input_variables=["raw_interview", "interview_problems_categories"]
# )

In [403]:
# print(answer_6.content)

<section_reflections>
1. Недостаток обучения по специализированным темам:
- Недостаток обучения по инвестициям в недвижимость, особенно в первичный рынок.

2. Неоптимальный формат обучения (большие группы):
- Большие группы на обучающих курсах (до 50 человек), что затрудняет эффективное усвоение материала.
- Предпочтение более компактных групп (до 15 человек) для лучшего взаимодействия и обратной связи.

3. Недостаток обучения по юридическим аспектам:
- Недостаточное освещение юридических вопросов, связанных с недвижимостью, в том числе налогообложение.
</section_reflections>

<section_results>
Анализ интервью показывает, что респондент имеет достаточно большой опыт работы в сфере недвижимости, но при этом сталкивается с определенными проблемами в процессе обучения и повышения квалификации:

1. Недостаток специализированных знаний по инвестициям в недвижимость, особенно на первичном рынке. Это является важным пробелом, так как данное направление становится все более востребованным.

2.

In [416]:
def processing_single_document(raw_interview:str = None):
    answer_1 = asks_llm(
        llm=llm_haiku, raw_data=[raw_interview],
        text_template="""
        Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
        
        Step-by-step instructions:
        1. Identify the main sections or phases of the interview, for example, introduction, general questions, main topics, concluding remarks. (In <section_main_phases> tags.)
        2. Break down each transcript into these sections, marking the beginning and end of each part. (In <section_transcripts> tags.)
        3. Create a standardized template for  <main_sections>structuring</main_sections> the transcripts consistently. (In <section_table> tags.)""",
        input_variables=["raw_interview"],
    )

    time.sleep(60)
    answer_2 = asks_llm(
        llm=llm_haiku, raw_data=[raw_interview],
        text_template="""
        Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
        
        Step-by-step instructions:
        1. Read through each interviews carefully and highlight or underline key phrases, words, or sentences that capture important ideas, concepts, or experiences. (In <section_key_phrases> tags.)
        2. Assign preliminary codes or labels to these highlighted segments, using short descriptive phrases or words that summarize the main idea. (In <section_codes> tags.)""",
        input_variables=["raw_interview"],
    )

    time.sleep(60)
    answer_2_section_codes = get_part_answer_llm(input_string=answer_2.content, tag="section_codes")
    answer_3 = asks_llm(
        llm=llm_haiku, raw_data=[raw_interview, answer_2_section_codes],
        text_template="""
        Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
        
        Step-by-step instructions:
        1. Look through the thematics codes <interview_code>{interview_code}</interview_code> and find patterns, similarities, or relationships among them. (In <section_relationships> tags.)
        2. Group related thematics codes together into broader categories or themes that capture the main topics discussed in the interviews. (In <section_group> tags.)
        3. Identify any sub-themes within each main theme to provide a more detailed understanding of the topics. (In <section_sub-themes> tags.)
        4. Create a diagram of the relationships between topics and subtopics to illustrate their relationship. (In <section_map> tags.)""",
        input_variables=["raw_interview", "interview_code"]
    )

    time.sleep(60)
    answer_4 = asks_llm(
        llm=llm_haiku, raw_data=[raw_interview],
        text_template="""
        Please read the interview provided to you carefully and follow the instructions below step by step: <interview>{raw_interview}</interview>
        
        Step-by-step instructions:
        1. Make a list of all the specific problems mentioned by the respondents. (In the <section_problems> tags)
        2. Count how many times each problem has been mentioned. (In the <section_problem_count> tags.)
        3. Create a table to display the frequency of occurrence of each problem in descending order. (In the <section_problem_table> tags.)
        4. Create a table of the frequency of occurrence of each problem by dividing the number of dimensions of the problem by the total number of problems. In the table, be sure to show the problem itself, how many times it occurs, its share relative to all problems, and the total number of problems. (In the <section_problem_table_normalize> tags).
        """,
        input_variables=["raw_interview"]
    )

    time.sleep(60)
    answer_4_section_problems = get_part_answer_llm(input_string=answer_4.content, tag="section_problems")
    answer_5 = asks_llm(
        llm=llm_haiku, raw_data=[raw_interview, answer_4_section_problems],
        text_template="""
        Please carefully read the interview provided to you and the problems identified in it. Follow the step-by-step instructions below: 
        <interview>{raw_interview}</interview> and <interview_problems>{interview_problems}</interview_problems> 
        
        Step-by-step instructions:
        1. Review the list of problems and look for commonalities or shared characteristics among them. (In <section_common_features> tags.)
        2. Develop a set of categories or problems types that can encompass the various issues mentioned. (In <section_problems_categories> tags.)
        3. Assignment each problem (<problems>) to one or more relevant categories. (In <section_map> tags.)
        4. Create a summary table or matrix showing the problems grouped by category. (In <section_result> tags.).
        """,
        input_variables=["raw_interview", "interview_problems"]
    )

    time.sleep(60)
    answer_5_section_problems_categories = get_part_answer_llm(input_string=answer_5.content, tag="section_problems_categories")
    answer_6 = asks_llm(
        llm=llm_haiku_t, raw_data=[raw_interview, answer_5_section_problems_categories],
        text_template="""
        Please carefully read the interview provided to you and the categories of problems identified in it. Follow the step-by-step instructions below: 
        <interview>{raw_interview}</interview> and <interview_problems>{interview_problems_categories}</interview_problems> 
        
        Step-by-step instructions:
        1. Analyze the topics, frequency of problems and their categories (<problems>). (In the <section_reflections> tags). 
        2. Analyze what these <reflection_sections> results say about the respondent's experience, views, or difficulties. (In the <section_results> tags). 
        3. Pay attention to any unexpected patterns, connections, or contrasts in the data. Share these observations. (In the <section_unexpected> tags).
        4. Develop preliminary explanations or hypotheses about why certain topics or issues have arisen and how they may relate to each other or to broader contextual factors. (In the <section_hypothesis> tags).
        5. Consider alternative explanations and counterexamples to clarify your hypothesis about <section_hypothesis>. (In the <section_alternatives> tags.)
        6. Identify areas where additional research or analysis may be needed to validate your ideas in the <section_alternatives>. (In the <section_additionals> tags).
        """,
        input_variables=["raw_interview", "interview_problems_categories"]
    )
    
    results = {
        "answer_1": answer_1.content,
        "answer_2": answer_2.content,
        "answer_3": answer_3.content,
        "answer_4": answer_4.content,
        "answer_5": answer_5.content,
        "answer_6": answer_6.content,
    }
    
    return results

In [417]:
results_interview_1 = processing_single_document(raw_interview=interview_1)

In [419]:
type(results_interview_1)

dict

In [420]:
# import json 

In [424]:
# with open("results_interview_1.json", "w", encoding='utf-8') as json_file:
#     json.dump(results_interview_1, json_file, ensure_ascii=False, indent=4, separators=(',', ': '))

In [426]:
print(results_interview_1["answer_1"])

<section_main_phases>
1. Введение и общие вопросы
2. Основные темы
3. Заключительные замечания
</section_main_phases>

<section_transcripts>
1. Введение и общие вопросы:
Давайте начнем тогда, наверное, с таких вот общих вопросов.

Светлана, расскажите, пожалуйста, вообще, сколько вам лет, да, вот где живете и как давно работаете в сфере риелдерства?

50, живу в Москве, работаю с 15-го года.

Я же правильно понимаю, что это ваше, скажем так, не основное образование, правильно, не основное род деятельности?

Ну, сейчас это мой основной род деятельности.

Образование, конечно, у меня другое, педагогическое было.

А расскажите, пожалуйста, почему?

Ну, это педагогическое, почему?

Ну, почему?

Нужны были деньги, я делала сама ипотеку.

И начала вначале, я параллельно занималась этой деятельностью, то есть вела педагогическую, руководитель, была руководителем отдела в пед. учреждении, и плюс еще занятия всякие проводила.

Учреждение дополнительного образования.

И параллельно у своей знаком